![UC logo](FCTUC_H_FundoClaro.png)

# JPEG Compressor and Decompressor

## Report + Code

__Done by:__
- Marco Pais - Nº 2019218
- Tiago Oliveira - Nº 2019219068
- Emílio - 

## Introduction

## Exercise 1

## Exercise 2

In [1]:
def encoder(img):  # 2
    # cm = getColormap() # 3.2
    # viewColormap(cm, img) # 3.3

    img = addPadding(img)  # 4.1
    
    R, G, B = separateRGB(img)  # 3.4
    viewChanels(R, G, B) # 3.5

    YCbCr = RGBtoYCbCr(R, G, B)  # 5
    showYCbCr(YCbCr) # 5
    
    global YBefore
    YBefore = YCbCr[:, :, 0].copy() # 10

    YD, CbD, CrD = downSample(YCbCr, dSample)  # 6
    showDownSample(YD, CbD, CrD) # 6

    Y_dct, Cb_dct, Cr_dct = calcDCT_8x8_64x64(YD, CbD, CrD)  # 7
    showDCT(Y_dct, Cb_dct, Cr_dct) # 7.1.2

    Y_Q, Cb_Q, Cr_Q = quantization(Y_dct, Cb_dct, Cr_dct) # 8
    showQuantization(Y_Q, Cb_Q, Cr_Q)

    Y_dcpm, Cb_dcpm, Cr_dcpm = DCPM(Y_Q, Cb_Q, Cr_Q) # 9
    showDCPM(Y_dcpm, Cb_dcpm, Cr_dcpm)

    return Y_dcpm, Cb_dcpm, Cr_dcpm

In [2]:
def decoder(Y_dcpm, Cb_dcpm, Cr_dcpm):  # 2
    Y_Q, Cb_Q, Cr_Q = IDCPM(Y_dcpm, Cb_dcpm, Cr_dcpm)

    Y_dct, Cb_dct, Cr_dct = deQuantization(Y_Q, Cb_Q, Cr_Q) # 8

    Y_enc, Cb_enc, Cr_enc = calcIDCT_8x8_64x64(Y_dct, Cb_dct, Cr_dct)  # 7

    YCbCrU = upSample(Y_enc, Cb_enc, Cr_enc, dSample)  # 6
    showUpSample(YCbCrU) # 6

    global YAfter
    YAfter = YCbCrU[:, :, 0].copy()

    RGBAfter = YCbCrtoRGB(YCbCrU)  # 5

    RGBAfter = unpadding(RGBAfter)  # 4

    return RGBAfter

## Exercise 3

In [3]:
def openImage():
    img = plt.imread(f'imagens/{img_name}.bmp')  # 3.1

    plt.figure(figsize=(5, 5))
    plt.title("Original Image")
    plt.imshow(img)
    plt.axis('off')

    plt.savefig(f"{img_name}/Original.png")

    return img

<table align="center">
<tr align="center">
<td> <h3> <strong> barn_mountains </strong> </h3> </td>
<td> <h3> <strong> peppers </strong> </h3> </td>
<td> <h3> <strong> logo </strong> </h3> </td>
</tr>
<td> <img src="barn_mountains/Original.png" /> </td>
<td> <img src="peppers/Original.png" /> </td>
<td> <img src="logo/Original.png" /> </td>
</table>

In [4]:
def getColormap():  # 3.2
    inp = str(input("Introduza colormap (r, g, b): "))
    r, g, b = inp.split(",")
    cm = c.LinearSegmentedColormap.from_list(
        'cm', [(0, 0, 0), (int(r), int(g), int(b))], N=256)
    return cm

In [5]:
def viewColormap(cm, img):  # 3.3
    plt.figure()
    plt.imshow(img, cm)

In [6]:
def separateRGB(img):  # 3.4
    R = img[:, :, 0]
    G = img[:, :, 1]
    B = img[:, :, 2]

    return R, G, B

In [7]:
def joinRGB(R, G, B):  # 3.4
    RGB = np.dstack((R, G, B))

    return RGB

In [8]:
def viewChanels(R, G, B):  # 3.5
    fig = plt.figure(figsize=(15, 5))
    plt.axis("off")
    fig.add_subplot(131)
    plt.title("Initial Red")
    plt.imshow(R, cmr)
    fig.add_subplot(132)
    plt.title("Initial Green")
    plt.imshow(G, cmg)
    fig.add_subplot(133)
    plt.title("Initial Blue")
    plt.imshow(B, cmb)

    plt.savefig(f"{img_name}/RGB.png")

<table align="center">
<tr align="center"> <td> <h3> <strong> barn_mountains </strong> </h3> </td> </tr>
<tr> <td> <img src="barn_mountains/RGB.png" /> </td> </tr>
</table>

<table align="center">
<tr align="center"> <td> <h3> <strong> peppers </strong> </h3> </td> </tr>
<tr> <td> <img src="peppers/RGB.png" /> </td> </tr>
</table>

<table align="center">
<tr align="center"> <td> <h3> <strong> logo </strong> </h3> </td> </tr>
<tr> <td> <img src="logo/RGB.png" /> </td> </tr>
</table>

## Exercise 4

In [9]:
def addPadding(img):  # 4.1
    h, w, _ = np.shape(img)
    if opt == 64:
        pad = 128
    else:
        pad = 16

    if h % pad != 0:
        add = pad - (h % pad)
        arr = np.ones(h, dtype=int)
        arr[h-1] = add + 1
        img = np.repeat(img, arr, axis=0)

    if w % pad != 0:
        add = pad - (w % pad)
        arr = np.ones(w, dtype=int)
        arr[w-1] = add + 1
        img = np.repeat(img, arr, axis=1)
    return img

In [10]:
def unpadding(RGBAfter):  # 4
    R, G, B = separateRGB(RGBAfter)

    R = R[:h, :w]
    G = G[:h, :w]
    B = B[:h, :w]

    return joinRGB(R, G, B)

In [11]:
def showRebuilt(RGBAfter):
    plt.figure(figsize=(5, 5))
    plt.title(f"Rebuilt Image - {quality}")
    plt.imshow(RGBAfter)
    plt.axis('off')

    plt.savefig(f"{img_name}/{quality}/Rebuilt_{quality}.png")

<div align="center">

<style> 
#logo img {
    margin-left: -5%;
    max-width: 130%;
    max-height: 130%;
    display: block;
}

img{
    max-width: 130%;
    max-height: 130%;
    display: block;
}
</style>

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr>
<td> <img src="barn_mountains/10/Rebuilt_10.png" /> </td>
<td> <img src="barn_mountains/25/Rebuilt_25.png" /> </td>
<td> <img src="barn_mountains/50/Rebuilt_50.png" /> </td> 
<td> <img src="barn_mountains/75/Rebuilt_75.png" /> </td>
<td> <img src="barn_mountains/100/Rebuilt_100.png" /> </td>
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr>
<td> <img src="peppers/10/Rebuilt_10.png" /> </td>
<td> <img src="peppers/25/Rebuilt_25.png" /> </td>
<td> <img src="peppers/50/Rebuilt_50.png" /> </td> 
<td> <img src="peppers/75/Rebuilt_75.png" /> </td>
<td> <img src="peppers/100/Rebuilt_100.png" /> </td>
</tr>
</table>
  
<table id="logo">
<h3> <strong> logo </strong> </h3>
<tr>
<td> <img src="logo/10/Rebuilt_10.png" /> </td>
<td> <img src="logo/25/Rebuilt_25.png" /> </td>
<td> <img src="logo/50/Rebuilt_50.png" /> </td> 
<td> <img src="logo/75/Rebuilt_75.png" /> </td>
<td> <img src="logo/100/Rebuilt_100.png" /> </td>
</tr>
</table>

</div>

## Exercise 5

In [12]:
def RGBtoYCbCr(R, G, B):  # 5
    matrix = np.array([[0.299, 0.587, 0.114],
                       [-0.168736, -0.331264, 0.5],
                       [0.5, -0.418688, -0.081312]])

    RGB = joinRGB(R, G, B)
    
    global RGBPadded
    RGBPadded = RGB.copy()

    YCbCr = RGB.dot(matrix.T)
    YCbCr[:, :, [1, 2]] += 128

    return YCbCr

In [13]:
def YCbCrtoRGB(YCbCr):  # 5
    matrix = np.array([[0.299, 0.587, 0.114],
                       [-0.168736, -0.331264, 0.5],
                       [0.5, -0.418688, -0.081312]])

    inverted = np.linalg.inv(matrix)
    YCbCr[:, :, [1, 2]] -= 128
    RGB = YCbCr.dot(inverted.T)
    RGB = RGB.round()
    RGB[RGB > 255] = 255
    RGB[RGB < 0] = 0
    RGB = RGB.astype(np.uint8)

    # comp = RGB == RGBPadded
    # res = comp.all()
    # print("Equal to original RGB?\n" + str(res))

    return RGB

In [14]:
def showYCbCr(YCbCr):  # 5
    Y, Cb, Cr = separateYCbCr(YCbCr)

    fig = plt.figure(figsize=(15, 5))
    plt.axis("off")
    fig.add_subplot(131)
    plt.title("Y Original")
    plt.imshow(Y, cmGray)
    fig.add_subplot(132)
    plt.title("Cb Original")
    plt.imshow(Cb, cmGray)
    fig.add_subplot(133)
    plt.title("Cr Original")
    plt.imshow(Cr, cmGray)

    plt.savefig(f"{img_name}/YCbCr.png")

<table align="center">
<tr align="center"> <td> <h3> barn_mountains </h3> </td> </tr>
<tr> <td> <img src="barn_mountains/YCbCr.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> peppers </h3> </td> </tr>
<tr> <td> <img src="peppers/YCbCr.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> logo </h3> </td> </tr>
<tr> <td> <img src="logo/YCbCr.png" /> </td> </tr>
</table>

In [15]:
def separateYCbCr(YCbCr):
    Y = YCbCr[:, :, 0]
    Cb = YCbCr[:, :, 1]
    Cr = YCbCr[:, :, 2]

    return Y, Cb, Cr

## Exercise 6

In [16]:
def downSample(YCbCr, dSample):  # 6
    Y, Cb, Cr = separateYCbCr(YCbCr)

    Cb = Cb[:, ::2]
    Cr = Cr[:, ::2]

    if(dSample == 420):
        Cb = Cb[::2, :]
        Cr = Cr[::2, :]

    return Y, Cb, Cr

In [17]:
def showDownSample(YD, CbD, CrD):  # 6
    fig = plt.figure(figsize=(15, 5))
    plt.axis("off")
    fig.add_subplot(131)
    plt.title("Y DownSampled")
    plt.imshow(YD, cmGray)
    fig.add_subplot(132)
    plt.title("Cb Downsampled")
    plt.imshow(CbD, cmGray)
    fig.add_subplot(133)
    plt.title("Cr Downsampled")
    plt.imshow(CrD, cmGray)

    plt.savefig(f"{img_name}/Downsample.png")

<table align="center">
<tr align="center"> <td> <h3> barn_mountains </h3> </td> </tr>
<tr> <td> <img src="barn_mountains/Downsample.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> peppers </h3> </td> </tr>
<tr> <td> <img src="peppers/Downsample.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> logo </h3> </td> </tr>
<tr> <td> <img src="logo/Downsample.png" /> </td> </tr>
</table>

In [18]:
def upSample(YD, CbD, CrD, dSample):  # 6
    CbU = np.repeat(CbD, 2, axis=1)
    CrU = np.repeat(CrD, 2, axis=1)

    if np.shape(YD)[0] % 2 != 0:
        CbU = np.delete(CbU, -1, 0)
        CrU = np.delete(CrU, -1, 0)

    if dSample == 420:
        CbU = np.repeat(CbU, 2, axis=0)
        CrU = np.repeat(CrU, 2, axis=0)

        if np.shape(YD)[1] % 2 != 0:
            CbU = np.delete(CbU, -1, 1)
            CrU = np.delete(CrU, -1, 1)

    YCbCrU = np.dstack((YD, CbU, CrU))

    return YCbCrU

In [19]:
def showUpSample(YCbCr):
    Y, Cb, Cr = separateYCbCr(YCbCr)

    fig = plt.figure(figsize=(15, 5))
    plt.axis("off")
    fig.add_subplot(131)
    plt.title(f"Y UpSampled - {quality}")
    plt.imshow(Y, cmGray)
    fig.add_subplot(132)
    plt.title(f"Cb UpSampled - {quality}")
    plt.imshow(Cb, cmGray)
    fig.add_subplot(133)
    plt.title(f"Cr UpSampled - {quality}")
    plt.imshow(Cr, cmGray)

    plt.savefig(f"{img_name}/{quality}/Upsample_{quality}.png")

<div align="center">

<style> 
#logo img {
    max-width: 129%;
    max-height: 129%;
    display: block;
}

img{
    max-width: 120%;
    max-height: 120%;
    display: block;
}
</style>

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="barn_mountains/10/Upsample_10.png" /> </td>
<td> <img src="barn_mountains/25/Upsample_25.png" /> </td>
<td> <img src="barn_mountains/50/Upsample_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="barn_mountains/75/Upsample_75.png" /> </td>
<td> <img src="barn_mountains/100/Upsample_100.png" /> </td>
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="peppers/10/Upsample_10.png" /> </td>
<td> <img src="peppers/25/Upsample_25.png" /> </td>
<td> <img src="peppers/50/Upsample_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="peppers/75/Upsample_75.png" /> </td>
<td> <img src="peppers/100/Upsample_100.png" /> </td>
</tr>
</table>

<table id="logo">
<h3> <strong> logo </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="logo/10/Upsample_10.png" /> </td>
<td> <img src="logo/25/Upsample_25.png" /> </td>
<td> <img src="logo/50/Upsample_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="logo/75/Upsample_75.png" /> </td>
<td> <img src="logo/100/Upsample_100.png" /> </td>
</tr>
</table>

</div>

## Exercise 7

In [20]:
def calcDCT_8x8_64x64(YD, CbD, CrD):  # 7
    if opt == 8 or opt == 64:
        Y_lines, Y_cols = np.shape(YD)
        Cb_lines, Cb_cols = np.shape(CbD)

        Y_dct = YD
        Cb_dct = CbD
        Cr_dct = CrD

        for i in range(0, Y_lines, opt):
            for j in range(0, Y_cols, opt):
                Y_dct[i:i+opt, j:j+opt] = dct(dct(Y_dct[i:i + opt,j:j + opt], norm="ortho").T, norm="ortho").T

        for i in range(0, Cb_lines, opt):
            for j in range(0, Cb_cols, opt):
                Cb_dct[i:i + opt, j:j + opt] = dct(dct(Cb_dct[i:i + opt, j:j + opt], norm="ortho").T, norm="ortho").T
                Cr_dct[i:i + opt, j:j + opt] = dct(dct(Cr_dct[i:i + opt, j:j + opt], norm="ortho").T, norm="ortho").T
    else:
        Y_dct = dct(dct(YD, norm='ortho').T, norm='ortho').T
        Cb_dct = dct(dct(CbD, norm='ortho').T, norm='ortho').T
        Cr_dct = dct(dct(CrD, norm='ortho').T, norm='ortho').T

    a = Y_dct.copy()
    b = Cb_dct.copy()
    c = Cr_dct.copy()

    """Y_enc, Cb_enc, Cr_enc = calcIDCT_8x8_64x64(Y_dct, Cb_dct, Cr_dct)
    comp = Y_enc == YD
    res = comp.all()
    print("Equal to original Y_d?\n" + str(res))
    comp = Cb_enc == CbD
    res = comp.all()
    print("Equal to original Cb_d?\n" + str(res))
    comp = Cr_enc == CrD
    res = comp.all()
    print("Equal to original Cr_d?\n" + str(res))"""
    
    return a, b, c

In [21]:
def showDCT(Y_dct, Cb_dct, Cr_dct):  # 7.1.2
    fig = plt.figure(figsize=(15, 5))
    fig.add_subplot(131)
    plt.title("Y_DCT - " + str(opt) + "x" + str(opt))
    plt.imshow(np.log(abs(Y_dct) + 0.0001), cmGray)
    fig.add_subplot(132)
    plt.title("Cb_DCT - " + str(opt) + "x" + str(opt))
    plt.imshow(np.log(abs(Cb_dct) + 0.0001), cmGray)
    fig.add_subplot(133)
    plt.title("Cr_DCT - " + str(opt) + "x" + str(opt))
    plt.imshow(np.log(abs(Cr_dct) + 0.0001), cmGray)

    plt.savefig(f"{img_name}/DCT{opt}x{opt}.png")

<table align="center">
<tr align="center"> <td> <h3> <strong> barn_mountains </strong> </h3> </td> </tr>
<tr> <td> <img src="barn_mountains/DCT8x8.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> <strong> peppers </strong> </h3> </td> </tr>
<tr> <td> <img src="peppers/DCT8x8.png" /> </td> </tr>
</table>
<table align="center">
<tr align="center"> <td> <h3> <strong> logo </strong> </h3> </td> </tr>
<tr> <td> <img src="logo/DCT8x8.png" /> </td> </tr>
</table>

In [22]:
def calcIDCT_8x8_64x64(Y_dct, Cb_dct, Cr_dct):  # 7
    if opt == 8 or opt == 64:
        Y_lines, Y_cols = np.shape(Y_dct)
        Cb_lines, Cb_cols = np.shape(Cb_dct)

        Y = Y_dct
        Cb = Cb_dct
        Cr = Cr_dct

        for i in range(0, Y_lines, opt):
            for j in range(0, Y_cols, opt):
                Y[i:i+opt, j:j+opt] = idct(idct(Y[i:i + opt,j:j + opt], norm="ortho").T, norm="ortho").T

        for i in range(0, Cb_lines, opt):
            for j in range(0, Cb_cols, opt):
                Cb[i:i + opt, j:j + opt] = idct(idct(Cb[i:i + opt, j:j + opt], norm="ortho").T, norm="ortho").T
                Cr[i:i + opt, j:j + opt] = idct(idct(Cr[i:i + opt, j:j + opt], norm="ortho").T, norm="ortho").T

    else:
        Y = idct(idct(Y_dct, norm='ortho').T, norm='ortho').T
        Cb = idct(idct(Cb_dct, norm='ortho').T, norm='ortho').T
        Cr = idct(idct(Cr_dct, norm='ortho').T, norm='ortho').T

    return Y, Cb, Cr

## Exercise 8

In [23]:
def getQuantizationMatrix():      # 8
    Q_Y = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
                    [12, 12, 14, 19, 26, 58, 60, 55],
                    [14, 13, 16, 24, 40, 57, 69, 56],
                    [14, 17, 22, 29, 51, 87, 80, 62],
                    [18, 22, 37, 56, 68, 109, 103, 77],
                    [24, 35, 55, 64, 81, 104, 113, 92],
                    [49, 64, 78, 87, 103, 121, 120, 101],
                    [72, 92, 95, 98, 112, 100, 103, 99]])

    Q_CbCr = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
                       [18, 21, 26, 66, 99, 99, 99, 99],
                       [24, 26, 56, 99, 99, 99, 99, 99],
                       [47, 66, 99, 99, 99, 99, 99, 99],
                       [99, 99, 99, 99, 99, 99, 99, 99],
                       [99, 99, 99, 99, 99, 99, 99, 99],
                       [99, 99, 99, 99, 99, 99, 99, 99],
                       [99, 99, 99, 99, 99, 99, 99, 99]])

    if quality == 100:
        Q_Y = np.ones(np.shape(Q_Y))
        Q_CbCr = np.ones(np.shape(Q_CbCr))
        return Q_Y, Q_CbCr

    elif quality > 50:
        Q_Y = (Q_Y * ((100 - quality) / 50)).round()
        Q_Y[Q_Y > 255] = 255
        Q_Y = Q_Y.astype(np.uint8)

        Q_CbCr = (Q_CbCr * ((100 - quality) / 50)).round()
        Q_CbCr[Q_CbCr > 255] = 255
        Q_CbCr = Q_CbCr.astype(np.uint8)
        return Q_Y, Q_CbCr
    else:
        Q_Y = (Q_Y * (50 / quality)).round()
        Q_Y[Q_Y > 255] = 255
        Q_Y = Q_Y.astype(np.uint8)

        Q_CbCr = (Q_CbCr * (50 / quality)).round()
        Q_CbCr[Q_CbCr > 255] = 255
        Q_CbCr = Q_CbCr.astype(np.uint8)
        return Q_Y, Q_CbCr

In [24]:
def quantization(Y_dct, Cb_dct, Cr_dct):    # 8
    Q_Y, Q_CbCr = getQuantizationMatrix()

    Y_lines, Y_cols = np.shape(Y_dct)
    Cb_lines, Cb_cols = np.shape(Cb_dct)

    YQ , CbQ, CrQ = Y_dct, Cb_dct, Cr_dct

    for i in range(0, Y_lines, opt):
        for j in range(0, Y_cols, opt):
            YQ[i:i + opt, j:j + opt] = np.round(YQ[i:i + opt, j:j + opt] / Q_Y)
            
    for i in range(0, Cb_lines, opt):
        for j in range(0, Cb_cols, opt):
            CbQ[i:i + opt, j:j + opt] = np.round(CbQ[i:i + opt, j:j + opt] / Q_CbCr)
            CrQ[i:i + opt, j:j + opt] = np.round(CrQ[i:i + opt, j:j + opt] / Q_CbCr)

    return YQ, CbQ, CrQ

In [25]:
def deQuantization(Y_Q, Cb_Q, Cr_Q):    # 8
    Q_Y, Q_CbCr = getQuantizationMatrix()

    Y_lines, Y_cols = np.shape(Y_Q)
    Cb_lines, Cb_cols = np.shape(Cb_Q)

    Y_dct, Cb_dct, Cr_dct = Y_Q, Cb_Q, Cr_Q

    for i in range(0, Y_lines, opt):
        for j in range(0, Y_cols, opt):
            Y_dct[i:i + opt, j:j + opt] = Y_dct[i:i + opt, j:j + opt] * Q_Y
                
    for i in range(0, Cb_lines, opt):
        for j in range(0, Cb_cols, opt):
            Cb_dct[i:i + opt, j:j + opt] = Cb_dct[i:i + opt, j:j + opt] * Q_CbCr
            Cr_dct[i:i + opt, j:j + opt] = Cr_dct[i:i + opt, j:j + opt] * Q_CbCr
    
    return Y_dct, Cb_dct, Cr_dct

In [26]:
def showQuantization(YQ, CbQ, CrQ):
    fig = plt.figure(figsize=(15, 5))
    fig.add_subplot(131)
    plt.title("Y_DCTQ - " + str(quality) + "%")
    plt.imshow(np.log(abs(YQ) + 0.0001), cmGray)
    fig.add_subplot(132)
    plt.title("Cb_DCTQ - " + str(quality) + "%")
    plt.imshow(np.log(abs(CbQ) + 0.0001), cmGray)
    fig.add_subplot(133)
    plt.title("Cr_DCTQ - " + str(quality) + "%")
    plt.imshow(np.log(abs(CrQ) + 0.0001), cmGray)    

    plt.savefig(f"{img_name}/{quality}/Quantization_{quality}.png")

<div align="center">

<style> 
#logo img {
    max-width: 129%;
    max-height: 129%;
    display: block;
}

img{
    max-width: 120%;
    max-height: 120%;
    display: block;
}
</style>

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="barn_mountains/10/Quantization_10.png" /> </td>
<td> <img src="barn_mountains/25/Quantization_25.png" /> </td>
<td> <img src="barn_mountains/50/Quantization_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="barn_mountains/75/Quantization_75.png" /> </td>
<td> <img src="barn_mountains/100/Quantization_100.png" /> </td>
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="peppers/10/Quantization_10.png" /> </td>
<td> <img src="peppers/25/Quantization_25.png" /> </td>
<td> <img src="peppers/50/Quantization_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="peppers/75/Quantization_75.png" /> </td>
<td> <img src="peppers/100/Quantization_100.png" /> </td>
</tr>
</table>

<table id="logo">
<h3> <strong> logo </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="logo/10/Quantization_10.png" /> </td>
<td> <img src="logo/25/Quantization_25.png" /> </td>
<td> <img src="logo/50/Quantization_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="logo/75/Quantization_75.png" /> </td>
<td> <img src="logo/100/Quantization_100.png" /> </td>
</tr>
</table>

</div>

## Exercise 9

In [27]:
def DCPM(YQ, CbQ, CrQ):  # 9
    Y_lines, Y_cols = np.shape(YQ)
    C_lines, C_cols = np.shape(CbQ)

    for i in range(0, Y_lines, 8):
        for j in range(0, Y_cols, 8):
            if (i == 0 and j == 0):
                dcY0 = YQ[0, 0]
                dcCb0 = CbQ[0, 0]
                dcCr0 = CrQ[0, 0]
                continue

            dcY = YQ[i, j]
            diffY = dcY - dcY0
            YQ[i, j] = diffY
            dcY0 = dcY
            if i < C_lines and j < C_cols:
                dcCb = CbQ[i, j]
                dcCr = CrQ[i, j]

                diffCb = dcCb - dcCb0
                diffCr = dcCr - dcCr0

                CbQ[i, j] = diffCb
                CrQ[i, j] = diffCr

                dcCb0 = dcCb
                dcCr0 = dcCr
    return YQ, CbQ, CrQ

In [28]:
def IDCPM(Y_dcpm, Cb_dcpm, Cr_dcpm):  # 9
    Y_lines, Y_cols = np.shape(Y_dcpm)
    C_lines, C_cols = np.shape(Cb_dcpm)

    YQ, CbQ, CrQ = Y_dcpm, Cb_dcpm, Cr_dcpm

    for i in range(0, Y_lines, 8):
        for j in range(0, Y_cols, 8):
            if i == 0 and j == 0:
                dcY0 = YQ[0, 0]
                dcCb0 = CbQ[0, 0]
                dcCr0 = CrQ[0, 0]
                continue

            diffY = YQ[i, j]
            diffY += dcY0
            YQ[i, j] = diffY
            dcY0 = diffY

            if i < C_lines and j < C_cols:
                diffCb = CbQ[i, j]
                diffCr = CrQ[i, j]

                diffCb += dcCb0
                diffCr += dcCr0

                CbQ[i, j] = diffCb
                CrQ[i, j] = diffCr

                dcCb0 = diffCb
                dcCr0 = diffCr

    return YQ, CbQ, CrQ

In [29]:
def showDCPM(Y_dcpm, Cb_dcpm, Cr_dcpm):
    fig = plt.figure(figsize=(15, 5))
    fig.add_subplot(131)
    plt.title("Y_DCPM - " + str(quality) + "%")
    plt.imshow(np.log(abs(Y_dcpm) + 0.0001), cmGray)
    fig.add_subplot(132)
    plt.title("Cb_DCPM - " + str(quality) + "%")
    plt.imshow(np.log(abs(Cb_dcpm) + 0.0001), cmGray)
    fig.add_subplot(133)
    plt.title("Cr_DCPM - " + str(quality) + "%")
    plt.imshow(np.log(abs(Cr_dcpm) + 0.0001), cmGray)  

    plt.savefig(f"{img_name}/{quality}/DCPM_{quality}.png")  

<div align="center">

<style> 
#logo img {
    max-width: 129%;
    max-height: 129%;
    display: block;
}

img{
    max-width: 120%;
    max-height: 120%;
    display: block;
}
</style>

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="barn_mountains/10/DCPM_10.png" /> </td>
<td> <img src="barn_mountains/25/DCPM_25.png" /> </td>
<td> <img src="barn_mountains/50/DCPM_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="barn_mountains/75/DCPM_75.png" /> </td>
<td> <img src="barn_mountains/100/DCPM_100.png" /> </td>
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="peppers/10/DCPM_10.png" /> </td>
<td> <img src="peppers/25/DCPM_25.png" /> </td>
<td> <img src="peppers/50/DCPM_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="peppers/75/DCPM_75.png" /> </td>
<td> <img src="peppers/100/DCPM_100.png" /> </td>
</tr>
</table>

<table id="logo">
<h3> <strong> logo </strong> </h3>
<tr style="margin-left:-5%;">
<td> <img src="logo/10/DCPM_10.png" /> </td>
<td> <img src="logo/25/DCPM_25.png" /> </td>
<td> <img src="logo/50/DCPM_50.png" /> </td>
</tr>
</table>
<table>
<tr>
<td> <img src="logo/75/DCPM_75.png" /> </td>
<td> <img src="logo/100/DCPM_100.png" /> </td>
</tr>
</table>

</div>

## Exercise 10

In [30]:
def YDiff():
    i = abs(np.subtract(YBefore, YAfter))

    plt.figure()
    plt.title(f"Y Diff - {quality}")
    plt.imshow(i, cmGray)
    plt.axis("off")

    plt.savefig(f"{img_name}/{quality}/YDiff_{quality}.png")

<div align="center">

<style> 
#logo img {
    margin-left:-10%;
    max-width: 130%;
    max-height: 130%;
    display: block;
}

img{
    margin-left:-5%;
    max-width: 150%;
    max-height: 150%;
    display: block;
}
</style>

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr>
<td> <img src="barn_mountains/10/YDiff_10.png" /> </td>
<td> <img src="barn_mountains/25/YDiff_25.png" /> </td>
<td> <img src="barn_mountains/50/YDiff_50.png" /> </td> 
<td> <img src="barn_mountains/75/YDiff_75.png" /> </td>
<td> <img src="barn_mountains/100/YDiff_100.png" /> </td>
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr>
<td> <img src="peppers/10/YDiff_10.png" /> </td>
<td> <img src="peppers/25/YDiff_25.png" /> </td>
<td> <img src="peppers/50/YDiff_50.png" /> </td> 
<td> <img src="peppers/75/YDiff_75.png" /> </td>
<td> <img src="peppers/100/YDiff_100.png" /> </td>
</tr>
</table>

<table id="logo">
<h3> <strong> logo </strong> </h3>
<tr>
<td> <img src="logo/10/YDiff_10.png" /> </td>
<td> <img src="logo/25/YDiff_25.png" /> </td>
<td> <img src="logo/50/YDiff_50.png" /> </td> 
<td> <img src="logo/75/YDiff_75.png" /> </td>
<td> <img src="logo/100/YDiff_100.png" /> </td>
</tr>
</table>

</div>

In [31]:
def MSE():
    return np.sum((RGBBefore - RGBAfter) ** 2) / (h * w)


def RMSE():
    return np.sqrt(MSE())


def SNR():
    p = np.sum((RGBBefore) ** 2) / (h * w)
    return 10 * np.log10(p / MSE())


def PSNR():
    return 10 * np.log10(RGBBefore.max()**2 / MSE())

<div align="center">

<table>
<h3> <strong> barn_mountains </strong> </h3>
<tr>
<td> <p> <strong> / </strong> </p> <td>
<td> <p> <strong> 10% </strong> </p> <td>
<td> <p> <strong> 25% </strong> </p> <td>
<td> <p> <strong> 50% </strong> </p> <td>
<td> <p> <strong> 75% </strong> </p> <td>
<td> <p> <strong> 100% </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> MSE </strong> </p> <td>
<td> <p> <strong> 745.1030218855219 </strong> </p> <td>
<td> <p> <strong> 430.2005723905724 </strong> </p> <td>
<td> <p> <strong> 293.21261784511785 </strong> </p> <td>
<td> <p> <strong> 187.50733164983166 </strong> </p> <td>
<td> <p> <strong> 55.53688552188552 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> RMSE </strong> </p> <td>
<td> <p> <strong> 27.29657527759704 </strong> </p> <td>
<td> <p> <strong> 20.74127701928144 </strong> </p> <td>
<td> <p> <strong> 17.123452275902714 </strong> </p> <td>
<td> <p> <strong> 13.693331649011926 </strong> </p> <td>
<td> <p> <strong> 7.4523073958261765 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> SNR </strong> </p> <td>
<td> <p> <strong> 18.464112560654293 </strong> </p> <td>
<td> <p> <strong> 20.849565969284978 </strong> </p> <td>
<td> <p> <strong> 22.514449254129616 </strong> </p> <td>
<td> <p> <strong> 24.456093272458475 </strong> </p> <td>
<td> <p> <strong> 29.740460597174266 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> PSNR </strong> </p> <td>
<td> <p> <strong> 19.408640361154117 </strong> </p> <td>
<td> <p> <strong> 21.7940937697848 </strong> </p> <td>
<td> <p> <strong> 23.45897705462944 </strong> </p> <td>
<td> <p> <strong> 25.400621072958298 </strong> </p> <td>
<td> <p> <strong> 30.68498839767409 </strong> </p> <td> 
</tr>
</table>

<table>
<h3> <strong> peppers </strong> </h3>
<tr>
<td> <p> <strong> / </strong> </p> <td>
<td> <p> <strong> 10% </strong> </p> <td>
<td> <p> <strong> 25% </strong> </p> <td>
<td> <p> <strong> 50% </strong> </p> <td>
<td> <p> <strong> 75% </strong> </p> <td>
<td> <p> <strong> 100% </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> MSE </strong> </p> <td>
<td> <p> <strong> 366.56260681152344 </strong> </p> <td>
<td> <p> <strong> 188.40497334798178 </strong> </p> <td>
<td> <p> <strong> 133.5154012044271 </strong> </p> <td>
<td> <p> <strong> 104.55508422851562 </strong> </p> <td>
<td> <p> <strong> 62.27764383951823 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> RMSE </strong> </p> <td>
<td> <p> <strong> 19.145824787966788 </strong> </p> <td>
<td> <p> <strong> 13.726069114935338 </strong> </p> <td>
<td> <p> <strong> 11.554886464367666 </strong> </p> <td>
<td> <p> <strong> 10.225218052859098 </strong> </p> <td>
<td> <p> <strong> 7.891618581730761 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> SNR </strong> </p> <td>
<td> <p> <strong> 19.284470568704638 </strong> </p> <td>
<td> <p> <strong> 22.17502854483714 </strong> </p> <td>
<td> <p> <strong> 23.670638520951673 </strong> </p> <td>
<td> <p> <strong> 24.732500609907483 </strong> </p> <td>
<td> <p> <strong> 26.98263043742454 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> PSNR </strong> </p> <td>
<td> <p> <strong> 22.489322005245675 </strong> </p> <td>
<td> <p> <strong> 25.37987998137818 </strong> </p> <td>
<td> <p> <strong> 26.875489957492707 </strong> </p> <td>
<td> <p> <strong> 27.937352046448517 </strong> </p> <td>
<td> <p> <strong> 30.187481873965577 </strong> </p> <td> 
</tr>
</table>

<table>
<h3> <strong> logo </strong> </h3>
<tr>
<td> <p> <strong> / </strong> </p> <td>
<td> <p> <strong> 10% </strong> </p> <td>
<td> <p> <strong> 25% </strong> </p> <td>
<td> <p> <strong> 50% </strong> </p> <td>
<td> <p> <strong> 75% </strong> </p> <td>
<td> <p> <strong> 100% </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> MSE </strong> </p> <td>
<td> <p> <strong> 207.8163487544484 </strong> </p> <td>
<td> <p> <strong> 100.93592882562278 </strong> </p> <td>
<td> <p> <strong> 80.99967259786477 </strong> </p> <td>
<td> <p> <strong> 60.74478291814947 </strong> </p> <td>
<td> <p> <strong> 43.0664128113879 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> RMSE </strong> </p> <td>
<td> <p> <strong> 14.41583673445452 </strong> </p> <td>
<td> <p> <strong> 10.046687455356755 </strong> </p> <td>
<td> <p> <strong> 8.999981810974107 </strong> </p> <td>
<td> <p> <strong> 7.793893950917568 </strong> </p> <td>
<td> <p> <strong> 6.56250049991525 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> SNR </strong> </p> <td>
<td> <p> <strong> 28.15532153179374 </strong> </p> <td>
<td> <p> <strong> 31.291660795504658 </strong> </p> <td>
<td> <p> <strong> 32.24728599905727 </strong> </p> <td>
<td> <p> <strong> 33.49702878986994 </strong> </p> <td>
<td> <p> <strong> 34.99073164366794 </strong> </p> <td> 
</tr>
<tr>
<td> <p> <strong> PSNR </strong> </p> <td>
<td> <p> <strong> 24.954006506854768 </strong> </p> <td>
<td> <p> <strong> 28.090345770565683 </strong> </p> <td>
<td> <p> <strong> 29.04597097411829 </strong> </p> <td>
<td> <p> <strong> 30.295713764930966 </strong> </p> <td>
<td> <p> <strong> 31.78941661872897 </strong> </p> <td> 
</tr>
</table>

</div>

## Main

In [32]:
import matplotlib.colors as c
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import dct, idct


def main():
    plt.close('all')

    global img_name, RGBBefore, RGBAfter, h, w, dSample, opt, quality

    img_name = "peppers"
    img = openImage()
    RGBBefore = img.copy()

    h, w, _ = np.shape(img)
    dSample = 420
    opt = 8
    #quality = 10

    Y_dcpm, Cb_dcpm, Cr_dcpm = encoder(img)
    RGBAfter = decoder(Y_dcpm, Cb_dcpm, Cr_dcpm)
    
    YDiff()
    
    showRebuilt(RGBAfter)
    plt.close("all")
        
    RGBBefore = RGBBefore.astype(float)
    RGBAfter = RGBAfter.astype(float)

    """print("Mean Square Error (MSE):", MSE())
    print("Root Mean Square Error (RMSE):", RMSE())
    print("Signal-to-Noise Ratio (SNR):", SNR())
    print("Peak Signal-to-Noise Ratio (PSNR):", PSNR())"""
    

if __name__ == '__main__':

    cmr = c.LinearSegmentedColormap.from_list(
        'cmr', [(0, 0, 0), (1, 0, 0)], N=256)
    cmg = c.LinearSegmentedColormap.from_list(
        'cmg', [(0, 0, 0), (0, 1, 0)], N=256)
    cmb = c.LinearSegmentedColormap.from_list(
        'cmb', [(0, 0, 0), (0, 0, 1)], N=256)
    cmGray = c.LinearSegmentedColormap.from_list(
        'cmGray', [(0, 0, 0), (1, 1, 1)], N=256)

    global quality
    for i in [10, 25, 50, 75, 100]:
        quality = i
        main()